In [7]:
# from PIL import Image
# from numpy import asarray
# import numpy as np

# # load the image
# image = Image.open('HA_3.png')
# # convert image to numpy array
# data = asarray(image)
# print(type(data))
# # summarize shape
# print(data.shape)

# # create Pillow image
# image2 = Image.fromarray(data)
# print(type(image2))

# # summarize image details
# print(image2.mode)
# print(image2.size)

# im = np.array(Image.open('HA_3.png').convert('L').resize((30,30))) #you can pass multiple arguments in single line
# print(type(im))

# Image.fromarray(im).save('HA_3_transformed.png')

# print(im)

import cv2 as cv
import numpy as np
from PIL import Image, ImageFilter, ImageEnhance
import PIL.ImageOps
import os
import glob
from sklearn.model_selection import train_test_split

def resize_only(file_name):
	img = Image.open(file_name).convert('LA')
	# img = img.filter(ImageFilter.SMOOTH)
	# img = img.filter(ImageFilter.SHARPEN)
	img = img.resize((28, 28))
	# img = PIL.ImageOps.invert(img)
	img.save("temp.png")
	img = cv.imread('temp.png', 0)
	img = cv.bitwise_not(img)
	img =  Image.fromarray(img)
	
	return img

def resize(file_name, invert):
	# image = cv.imread('%s' % file_name,0)
	img = Image.open(file_name).convert('LA')
# 	if invert==True:
# 		img = PIL.ImageOps.invert(img)
	enhancer = ImageEnhance.Contrast(img)
	
	factor = 2.0
	img = enhancer.enhance(factor)
	img = img.filter(ImageFilter.SMOOTH)
	img = img.filter(ImageFilter.SHARPEN)
	img = img.resize((28, 28))
	# print(type(img))    
    
	img.save("temp.png")
    
	img = cv.imread('temp.png', 0)
    
	if invert==True:
		img = cv.bitwise_not(img)
	ret,th = cv.threshold(img,0,255,cv.THRESH_BINARY+cv.THRESH_OTSU)
    
	blur = cv.GaussianBlur(img,(5,5),0)
	ret2,th2 = cv.threshold(blur,0,255,cv.THRESH_BINARY+cv.THRESH_OTSU)
    
	mask = np.where(th ==  0, th2, 255)
    
	img = np.where(mask == 0, img, 255)

#     create a CLAHE object (Arguments are optional)
	clahe = cv.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
	img = clahe.apply(img)

	_,img_remove = cv.threshold(img,95,255,cv.THRESH_TOZERO)
	img_remove = cv.bitwise_not(img_remove)
	_,img = cv.threshold(img_remove,95,255,cv.THRESH_TOZERO)

# 	img = cv.bitwise_not(img)
    
	img =  Image.fromarray(img)
  
	return img
	# img.save("resized_"+file_name)

def preprocess():
	# crop(str(sys.argv))
	script_dir = os.path.abspath('')
	i=["A","BA","CA","DA","GA","HA","I","JA","KA","LA","MA","NA","NGA","NYA","PA","PANELENG","PANEULEUNG","PANGHULU","PANGLAYAR","PANOLONG","PANYUKU","PATEN","RA","SA","TA","U","WA","YA"]
	j=[30,47,19,67,37,27,16,21,60,60,56,120,25,14,61,42,63,60,23,35,36,84,56,90,78,18,24,22]
	k=[12,30,7,45,16,11,7,9,40,25,24,80,10,6,39,28,42,40,10,24,24,36,36,60,52,7,10,9]
	ganed = [0,2,4,5,6,7,12,13,18,19,20,25,26,27]

	final_np = np.array([])
	final_label = np.array([])

    #   train_image to ready_to_train
	for x in range(0, len(i), 1):
		sumImg = 0
		train_directory_path = os.path.join(script_dir,"train-test_image","dataset2")
		train_count_image = len(glob.glob1(train_directory_path,"%s_*.png" % (i[x])))

		print("Aksara = " + str(i[x]))
		print("Count_image training = " + str(train_count_image))

		for y in range(1, train_count_image+1):
			abs_file_path = os.path.join(directory_path, "%s_%s.png" % (i[aksara],str(y)))
			img = Image.open(abs_file_path)
			img = np.array(img)
			img = img[:, :]
			final_np = np.append(final_np,img)
			final_label = np.append(final_label,aksara)

# 			rel_path = "train_image/%s_%s.png" % (i[x],str(y))
# 			abs_file_path = os.path.join(script_dir, rel_path)
            
# 			img = resize(abs_file_path, invert=False)
# 			img.save("train-test_image/%s_%s.png" % (i[x],str(y)))
# 			# img = resize(abs_file_path, invert=False)
# 			# img.save("ready_to_train_3/%s_%s.png" % (i[x],str(y)))
		sumImg = sumImg + train_count_image
		np.random.seed(42)

		test_directory_path = os.path.join(script_dir,"test_image/%s" % i[x])
		test_count_image = len(glob.glob1(test_directory_path,"%s_*.png" % (i[x])))

		for y in range(1, test_count_image+1):
			rel_path = os.path.join(test_directory_path,"%s_%s.png" % (i[x],str(y)))
			abs_file_path = os.path.join(script_dir, rel_path)
            
			img = resize(abs_file_path, invert=False)
			img.save("train-test_image/%s_%s.png" % (i[x],str(y+sumImg)))
		sumImg = sumImg + test_count_image

# 		if x == 0:
# 			for y in range(1, sumImg+1):
# 				rel_path = os.path.join("train-test_image","%s_%s.png" % (i[x],str(y)))
# 				abs_file_path = os.path.join(script_dir, rel_path)

# 				img = Image.open(abs_file_path)
# 				img = np.array(img)
# 				img = img[:, :]
# 				final_np = np.append(final_np,img)
# 				final_label = np.append(final_label,x)
# 			final_np = final_np.reshape((sumImg, 28, 28, 1)).astype(np.float32)
# 			final_label = final_label.reshape((sumImg)).astype(np.int32)
# 			data_train, data_test, labels_train, labels_test = train_test_split(final_np, final_label, test_size=0.30, random_state=42)
# 			print(final_label)
# 			print(data_train.shape)
# 			print(data_test.shape)
# 			print(labels_train)
# 			print(labels_test)
            
# #   GAN_image to ready_to_train
# 	for x in ganed:
# 		for y in range(1, 26):
# 			rel_path = "GAN_generated_images/%s_%s.png" % (i[x],str(y+j[x]))
# # 			rel_path = "ready_to_train/%s_%s.png" % (i[x],str(y))
# 			abs_file_path = os.path.join(script_dir, rel_path)
            
# 			img = resize(abs_file_path, invert=True)
# 			img.save("ready_to_train_3/%s_%s.png" % (i[x],str(y+j[x])))

# #   test_image to ready_to_test (validation)
# 	sumImg = 0
# 	final_np = np.array([])
# 	final_label = np.array([])
# 	for x in range(0, len(i), 1):
# 		sumImg = sumImg + k[x]
# 		for y in range(1, int((k[x])/2)+1):
# 			rel_path = "test_image/%s/%s_%s.png" % (i[x],i[x],str(y))
# 			# rel_path = "ready_to_train/%s_%s.png" % (i[x],str(y))
# 			abs_file_path = os.path.join(script_dir, rel_path)

# 			# img = resize(abs_file_path, invert=False)
# 			# img.save("ready_to_test_2/%s_%s.png" % (i[x],str(y)))
# 			img = resize_only(abs_file_path)
# 			img.save("ready_to_test/%s_%s.png" % (i[x],str(y)))

# # 	test_image to ready_to_test (test)
# 	sumImg = 0
# 	for x in range(0, len(i), 1):
# 		sumImg = sumImg - (-k[x]//2)
# 		for y in range((-(-k[x])//2)+1, k[x]+1):
# #             rel_path = "test_image/%s/%s_%s.png" % (i[x],i[x],str(y))
# 			rel_path = "test_image/%s/%s_%s.png" % (i[x],i[x],str(y))
# 			# rel_path = "ready_to_train/%s_%s.png" % (i[x],str(y))
# 			abs_file_path = os.path.join(script_dir, rel_path)
            
# 			img = Image.open(abs_file_path)
# 			img = np.array(img)
# 			img = img[:, :, 0]
# 			img = resize_only(abs_file_path)
# 			img.save("ready_to_test/%s_%s.png" % (i[x],str(y)))
# 			# img = resize(abs_file_path, invert=False)
# 			# img.save("ready_to_test_2/%s_%s.png" % (i[x],str(y)))

# if __name__ == "__main__":
# 	main()



In [8]:
preprocess()

In [1]:
def get_sunda(aksara):
    script_dir = os.path.abspath('')
    i=["A","BA","CA","DA","GA","HA","I","JA","KA","LA","MA","NA","NGA","NYA","PA","PANELENG","PANEULEUNG","PANGHULU","PANGLAYAR","PANOLONG","PANYUKU","PATEN","RA","SA","TA","U","WA","YA"]
    j=[30,47,19,67,37,27,16,21,60,60,56,120,25,14,61,42,63,60,23,35,36,84,56,90,78,18,24,22]
    k=[12,30,7,45,16,11,7,9,40,25,24,80,10,6,39,28,42,40,10,24,24,36,36,60,52,7,10,9]
    ganed = [0,2,4,5,6,7,12,13,18,19,20,25,26,27]

    final_np = np.array([])
    final_label = np.array([])

    #   train_image to ready_to_train
#for x in range(0, len(i), 1):
    sumImg = 0
    directory_path = os.path.join(script_dir,"train-test_image","dataset2")
    count_image = len(glob.glob1(directory_path,"%s_*.png" % (i[aksara])))

    for y in range(1, count_image+1):
        abs_file_path = os.path.join(directory_path, "%s_%s.png" % (i[aksara],str(y)))
        img = Image.open(abs_file_path)
        img = np.array(img)
        img = img[:, :]
        final_np = np.append(final_np,img)
        final_label = np.append(final_label,aksara)
    sumImg = sumImg + count_image
    
    directory_path = os.path.join(script_dir,"train-test_image","koropak_28")
    count_image = len(glob.glob1(directory_path,"%s_*.png" % (i[aksara])))
    print("Count_image koropak = " + str(count_image))

    for y in range(0, count_image):
        abs_file_path = os.path.join(directory_path, "%s_%s.png" % (i[aksara],str(y)))
        img = Image.open(abs_file_path)
        img = np.array(img)
        img = img[:, :]
        final_np = np.append(final_np,img)
        final_label = np.append(final_label,aksara)
    sumImg = sumImg + count_image

    directory_path = os.path.join(script_dir,"train-test_image","kawih_p")
    count_image = len(glob.glob1(directory_path,"%s_*.png" % (i[aksara])))
    print("Count_image kawih = " + str(count_image))

    for y in range(0, count_image):
        abs_file_path = os.path.join(directory_path, "%s_%s.png" % (i[aksara],str(y)))
        img = Image.open(abs_file_path)
        img = np.array(img)
        img = img[:, :]
        final_np = np.append(final_np,img)
        final_label = np.append(final_label,aksara)
    sumImg = sumImg + count_image

    print("SumImg = " + str(sumImg))

    final_np = final_np.reshape((sumImg, 28, 28, 1)).astype(np.float32)
    final_label = final_label.reshape((sumImg)).astype(np.int32)

#     final_np = final_np.reshape((sumImg, 28, 28, 1)).astype(np.float32)
#     final_label = final_label.reshape((sumImg)).astype(np.int32)
    data_train, data_test, labels_train, labels_test = train_test_split(final_np, final_label, test_size=0.30, random_state=42)
    return(data_train,sumImg)


#     print(final_label)
#     print(data_train.shape)
#     print(data_test.shape)
#     print(labels_train)
#     print(labels_test)

In [2]:
from __future__ import print_function, division

from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam

import matplotlib.pyplot as plt

import sys

import numpy as np
import cv2 as cv
import os
import scipy
import tensorflow as tf
from PIL import Image
import glob
from sklearn.model_selection import train_test_split
import time

class GAN():
    def __init__(self):
        self.img_rows = 28
        self.img_cols = 28
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.latent_dim = 100

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy',
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise as input and generates imgs
        z = Input(shape=(self.latent_dim,))
        img = self.generator(z)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The discriminator takes generated images as input and determines validity
        validity = self.discriminator(img)

        # The combined model  (stacked generator and discriminator)
        # Trains the generator to fool the discriminator
        self.combined = Model(z, validity)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)


    def build_generator(self):

        model = Sequential()

        model.add(Dense(256, input_dim=self.latent_dim))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(1024))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(np.prod(self.img_shape), activation='tanh'))
        model.add(Reshape(self.img_shape))

        model.summary()

        noise = Input(shape=(self.latent_dim,))
        img = model(noise)

        return Model(noise, img)

    def build_discriminator(self):

        model = Sequential()

        model.add(Flatten(input_shape=self.img_shape))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(256))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(1, activation='sigmoid'))
        model.summary()

        img = Input(shape=self.img_shape)
        validity = model(img)

        return Model(img, validity)

    def train(self, aksara, aksara_num, epochs, batch_size=128, sample_interval=50):

        # Load the dataset
#         preprocess()
        X_train, sumTrain= get_sunda(aksara)
        X_train = X_train[:,:,:,0]
#         (X_train, _), (_, _) = mnist.load_data()

        # Rescale -1 to 1
        X_train = X_train / 127.5 - 1.
        X_train = np.expand_dims(X_train, axis=3)

        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))
        
        min_loss = 100

        for epoch in range(epochs):

            # ---------------------
            #  Train Discriminator
            # ---------------------

            # Select a random batch of images
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            imgs = X_train[idx]

            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))

            # Generate a batch of new images
            gen_imgs = self.generator.predict(noise)

            # Train the discriminator
            d_loss_real = self.discriminator.train_on_batch(imgs, valid)
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            # ---------------------
            #  Train Generator
            # ---------------------

            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))

            # Train the generator (to have the discriminator label samples as valid)
            g_loss = self.combined.train_on_batch(noise, valid)

            # Plot the progress
#             print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

            # If at save interval => save generated image samples
            if epoch % sample_interval == 0:
                print ("%d-%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (aksara, epoch, d_loss[0], 100*d_loss[1], g_loss))
                self.sample_images(epoch, aksara, sumTrain, aksara_num)
            
#             if d_loss[0] + g_loss < min_loss and epoch > 10000:
#                 min_loss = d_loss[0] + g_loss
#                 self.sample_images(epoch, aksara, sumTrain, aksara_num)

    def sample_images(self, epoch, aksara, sumTrain, aksara_num):
        aksaraList=["A","BA","CA","DA","GA","HA","I","JA","KA","LA","MA","NA","NGA","NYA","PA","PANELENG","PANEULEUNG","PANGHULU","PANGLAYAR","PANOLONG","PANYUKU","PATEN","RA","SA","TA","U","WA","YA"]
        r, c = 10, 10
        noise = np.random.normal(0, 1, (r * c, self.latent_dim))
        gen_imgs = self.generator.predict(noise)

        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5
        for i in range(0,aksara_num):
            img = gen_imgs[i,:,:,0]
            normalizedImg = np.zeros((28,28))
            normalizedImg = cv.normalize(img, normalizedImg, 0, 255, cv.NORM_MINMAX)
            cv.imwrite("GAN_generated_images/GAN_training_only/%s_%s.png" % (aksaraList[aksara],str(sumTrain+i+1)), normalizedImg)
        for i in range(aksara_num,aksara_num+10):
            img = gen_imgs[i,:,:,0]
            normalizedImg = np.zeros((28,28))
            normalizedImg = cv.normalize(img, normalizedImg, 0, 255, cv.NORM_MINMAX)
            cv.imwrite("GAN_generated_images/GAN_training_only/%s_Cadangan_%s.png" % (aksaraList[aksara],str(i+1)), normalizedImg)

if __name__ == '__main__':
    start_time = time.time()
    aksara=[13,18,25,27]
    aksara_num=[38,23,37,25]
    for i in range(0,4):
        gan = GAN()
        gan.train(aksara[i], aksara_num[i], epochs=10001, batch_size=24, sample_interval=1000)
        print("-----------------------------------------------------------------")
        print("--- %s seconds ---" % (time.time() - start_time))
        print("-----------------------------------------------------------------")
        

Using TensorFlow backend.
C:\Users\Iyo\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Iyo\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Iyo\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Iyo\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:529: Fu

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               401920    
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               131328    
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 256)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 257       
Total params: 533,505
Trainable params: 533,505
Non-trainable params: 

C:\Users\Iyo\Anaconda3\lib\site-packages\keras\engine\training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


13-0 [D loss: 0.938999, acc.: 27.08%] [G loss: 0.683352]
13-1000 [D loss: 0.266018, acc.: 91.67%] [G loss: 2.932680]
13-2000 [D loss: 0.263276, acc.: 89.58%] [G loss: 3.256689]
13-3000 [D loss: 0.189221, acc.: 91.67%] [G loss: 2.358646]
13-4000 [D loss: 0.233752, acc.: 91.67%] [G loss: 2.876971]
13-5000 [D loss: 0.188877, acc.: 93.75%] [G loss: 2.793161]
13-6000 [D loss: 0.208905, acc.: 91.67%] [G loss: 2.608845]
13-7000 [D loss: 0.485863, acc.: 79.17%] [G loss: 3.060521]
13-8000 [D loss: 0.270975, acc.: 87.50%] [G loss: 2.298813]
13-9000 [D loss: 0.233233, acc.: 91.67%] [G loss: 2.227478]
13-10000 [D loss: 0.198722, acc.: 93.75%] [G loss: 2.763816]
-----------------------------------------------------------------
--- 736.6054253578186 seconds ---
-----------------------------------------------------------------
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_2 (Flatten)          (None, 784)    

In [4]:

if __name__ == '__main__':
    start_time = time.time()
    aksara=[0]
    aksara_num=[16]
    for i in range(0,1):
        gan = GAN()
        gan.train(aksara[i], aksara_num[i], epochs=30001, batch_size=48, sample_interval=1000)
        print("-----------------------------------------------------------------")
        print("--- %s seconds ---" % (time.time() - start_time))
        print("-----------------------------------------------------------------")
        

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_2 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 512)               401920    
_________________________________________________________________
leaky_re_lu_6 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 256)               131328    
_________________________________________________________________
leaky_re_lu_7 (LeakyReLU)    (None, 256)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 1)                 257       
Total params: 533,505
Trainable params: 533,505
Non-trainable params: 0
_________________________________________________________________
____

C:\Users\Sadrakh\AppData\Local\Continuum\anaconda3\lib\site-packages\keras\engine\training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


0-0 [D loss: 0.798397, acc.: 51.04%] [G loss: 0.718805]
0-1000 [D loss: 0.134721, acc.: 100.00%] [G loss: 3.232478]
0-2000 [D loss: 0.233812, acc.: 93.75%] [G loss: 2.447292]
0-3000 [D loss: 0.245962, acc.: 90.62%] [G loss: 2.709725]
0-4000 [D loss: 0.197391, acc.: 92.71%] [G loss: 2.442304]
0-5000 [D loss: 0.267694, acc.: 88.54%] [G loss: 2.841992]
0-6000 [D loss: 0.170154, acc.: 93.75%] [G loss: 2.776485]
0-7000 [D loss: 0.233312, acc.: 87.50%] [G loss: 2.457732]
0-8000 [D loss: 0.164447, acc.: 94.79%] [G loss: 2.848845]
0-9000 [D loss: 0.098829, acc.: 97.92%] [G loss: 3.764603]
0-10000 [D loss: 0.224366, acc.: 92.71%] [G loss: 3.156589]
0-11000 [D loss: 0.166608, acc.: 95.83%] [G loss: 3.200519]
0-12000 [D loss: 0.139632, acc.: 96.88%] [G loss: 3.521345]
0-13000 [D loss: 0.172081, acc.: 93.75%] [G loss: 3.368053]
0-14000 [D loss: 0.081057, acc.: 97.92%] [G loss: 3.755254]
0-15000 [D loss: 0.190834, acc.: 94.79%] [G loss: 3.514821]
0-16000 [D loss: 0.154563, acc.: 94.79%] [G loss: 3.

In [3]:

if __name__ == '__main__':
    start_time = time.time()
    aksara=[1,2]
    aksara_num=[24,24]
    for i in range(0,2):
        gan = GAN()
        gan.train(aksara[i], aksara_num[i], epochs=30001, batch_size=48, sample_interval=1000)
        print("-----------------------------------------------------------------")
        print("--- %s seconds ---" % (time.time() - start_time))
        print("-----------------------------------------------------------------")
        

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               401920    
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               131328    
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 256)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 257       
Total params: 533,505
Trainable params: 533,505
Non-trainable params: 

C:\Users\Sadrakh\AppData\Local\Continuum\anaconda3\lib\site-packages\keras\engine\training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


1-0 [D loss: 0.626050, acc.: 45.83%] [G loss: 0.485649]
1-1000 [D loss: 0.128476, acc.: 100.00%] [G loss: 4.069482]
1-2000 [D loss: 0.237716, acc.: 91.67%] [G loss: 3.169679]
1-3000 [D loss: 0.054870, acc.: 100.00%] [G loss: 3.807199]
1-4000 [D loss: 0.132430, acc.: 94.79%] [G loss: 3.183053]
1-5000 [D loss: 0.107382, acc.: 96.88%] [G loss: 3.201871]
1-6000 [D loss: 0.260991, acc.: 89.58%] [G loss: 2.767683]
1-7000 [D loss: 0.275476, acc.: 91.67%] [G loss: 2.770325]
1-8000 [D loss: 0.168208, acc.: 94.79%] [G loss: 3.542899]
1-9000 [D loss: 0.142599, acc.: 93.75%] [G loss: 3.440157]
1-10000 [D loss: 0.186412, acc.: 93.75%] [G loss: 3.363034]
1-11000 [D loss: 0.142329, acc.: 94.79%] [G loss: 3.309966]
1-12000 [D loss: 0.118339, acc.: 95.83%] [G loss: 3.870631]
1-13000 [D loss: 0.173666, acc.: 93.75%] [G loss: 3.689058]
1-14000 [D loss: 0.156913, acc.: 93.75%] [G loss: 3.553060]
1-15000 [D loss: 0.186420, acc.: 92.71%] [G loss: 3.478856]
1-16000 [D loss: 0.190914, acc.: 89.58%] [G loss: 3

In [ ]:
%%time
aksara=2
aksara_num=27
gan = GAN()
gan.train(aksara, aksara_num, epochs=20001, batch_size=48, sample_interval=1000)
print("-----------------------------------------------------------------")

In [3]:

if __name__ == '__main__':
    start_time = time.time()
    aksara=[4,5,6]
    aksara_num=[8,19,24]
    for i in range(0,3):
        gan = GAN()
        gan.train(aksara[i], aksara_num[i], epochs=30001, batch_size=48, sample_interval=1000)
        print("-----------------------------------------------------------------")
        print("--- %s seconds ---" % (time.time() - start_time))
        print("-----------------------------------------------------------------")
        

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               401920    
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               131328    
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 256)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 257       
Total params: 533,505
Trainable params: 533,505
Non-trainable params: 

C:\Users\Sadrakh\AppData\Local\Continuum\anaconda3\lib\site-packages\keras\engine\training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


4-0 [D loss: 0.610326, acc.: 72.92%] [G loss: 0.830040]
4-1000 [D loss: 0.029783, acc.: 100.00%] [G loss: 4.822430]
4-2000 [D loss: 0.149972, acc.: 94.79%] [G loss: 3.693342]
4-3000 [D loss: 0.127729, acc.: 94.79%] [G loss: 3.944894]
4-4000 [D loss: 0.325468, acc.: 88.54%] [G loss: 2.532356]
4-5000 [D loss: 0.327141, acc.: 87.50%] [G loss: 2.639507]
4-6000 [D loss: 0.196871, acc.: 92.71%] [G loss: 2.968440]
4-7000 [D loss: 0.122801, acc.: 95.83%] [G loss: 3.039479]
4-8000 [D loss: 0.189742, acc.: 93.75%] [G loss: 2.867384]
4-9000 [D loss: 0.205211, acc.: 93.75%] [G loss: 3.190418]
4-10000 [D loss: 0.232591, acc.: 89.58%] [G loss: 3.439430]
4-11000 [D loss: 0.140849, acc.: 96.88%] [G loss: 3.305743]
4-12000 [D loss: 0.115760, acc.: 95.83%] [G loss: 4.155413]
4-13000 [D loss: 0.280250, acc.: 89.58%] [G loss: 3.961958]
4-14000 [D loss: 0.110069, acc.: 97.92%] [G loss: 3.959473]
4-15000 [D loss: 0.199278, acc.: 92.71%] [G loss: 3.896241]
4-16000 [D loss: 0.194762, acc.: 90.62%] [G loss: 3.

In [3]:

if __name__ == '__main__':
    start_time = time.time()
    aksara=[7,12,13]
    aksara_num=[24,21,24]
    for i in range(0,3):
        gan = GAN()
        gan.train(aksara[i], aksara_num[i], epochs=30001, batch_size=48, sample_interval=1000)
        print("-----------------------------------------------------------------")
        print("--- %s seconds ---" % (time.time() - start_time))
        print("-----------------------------------------------------------------")
        

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               401920    
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               131328    
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 256)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 257       
Total params: 533,505
Trainable params: 533,505
Non-trainable params: 

C:\Users\Sadrakh\AppData\Local\Continuum\anaconda3\lib\site-packages\keras\engine\training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


7-0 [D loss: 0.596523, acc.: 50.00%] [G loss: 0.392304]
7-1000 [D loss: 0.396122, acc.: 84.38%] [G loss: 2.487470]
7-2000 [D loss: 0.342031, acc.: 85.42%] [G loss: 1.963295]
7-3000 [D loss: 0.235858, acc.: 91.67%] [G loss: 1.995665]
7-4000 [D loss: 0.290153, acc.: 88.54%] [G loss: 2.245050]
7-5000 [D loss: 0.247665, acc.: 90.62%] [G loss: 1.985743]
7-6000 [D loss: 0.303459, acc.: 87.50%] [G loss: 2.142445]
7-7000 [D loss: 0.307098, acc.: 87.50%] [G loss: 2.406739]
7-8000 [D loss: 0.268866, acc.: 89.58%] [G loss: 2.649334]
7-9000 [D loss: 0.260654, acc.: 92.71%] [G loss: 3.226983]
7-10000 [D loss: 0.290214, acc.: 80.21%] [G loss: 2.393304]
7-11000 [D loss: 0.306444, acc.: 85.42%] [G loss: 3.395325]
7-12000 [D loss: 0.198541, acc.: 91.67%] [G loss: 3.258292]
7-13000 [D loss: 0.291292, acc.: 88.54%] [G loss: 2.990735]
7-14000 [D loss: 0.200564, acc.: 91.67%] [G loss: 3.006142]
7-15000 [D loss: 0.206361, acc.: 91.67%] [G loss: 4.148321]
7-16000 [D loss: 0.157281, acc.: 92.71%] [G loss: 5.8

In [3]:

if __name__ == '__main__':
    start_time = time.time()
    aksara=[18,25]
    aksara_num=[22,24]
    for i in range(0,2):
        gan = GAN()
        gan.train(aksara[i], aksara_num[i], epochs=30001, batch_size=48, sample_interval=1000)
        print("-----------------------------------------------------------------")
        print("--- %s seconds ---" % (time.time() - start_time))
        print("-----------------------------------------------------------------")
        

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               401920    
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               131328    
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 256)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 257       
Total params: 533,505
Trainable params: 533,505
Non-trainable params: 

C:\Users\Sadrakh\AppData\Local\Continuum\anaconda3\lib\site-packages\keras\engine\training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


18-0 [D loss: 0.504987, acc.: 69.79%] [G loss: 0.542743]
18-1000 [D loss: 0.281308, acc.: 88.54%] [G loss: 2.455142]
18-2000 [D loss: 0.267303, acc.: 88.54%] [G loss: 2.400737]
18-3000 [D loss: 0.281740, acc.: 84.38%] [G loss: 2.304826]
18-4000 [D loss: 0.361859, acc.: 85.42%] [G loss: 1.952903]
18-5000 [D loss: 0.319918, acc.: 87.50%] [G loss: 2.132948]
18-6000 [D loss: 0.268459, acc.: 89.58%] [G loss: 2.359825]
18-7000 [D loss: 0.293484, acc.: 87.50%] [G loss: 2.292262]
18-8000 [D loss: 0.427897, acc.: 78.12%] [G loss: 3.644894]
18-9000 [D loss: 0.208218, acc.: 89.58%] [G loss: 3.089723]
18-10000 [D loss: 0.234326, acc.: 91.67%] [G loss: 2.638286]
18-11000 [D loss: 0.241724, acc.: 87.50%] [G loss: 3.713802]
18-12000 [D loss: 0.241218, acc.: 88.54%] [G loss: 3.777828]
18-13000 [D loss: 0.156676, acc.: 93.75%] [G loss: 4.592323]
18-14000 [D loss: 0.166597, acc.: 91.67%] [G loss: 4.373890]
18-15000 [D loss: 0.218427, acc.: 88.54%] [G loss: 3.575727]
18-16000 [D loss: 0.185256, acc.: 90.

In [3]:

if __name__ == '__main__':
#     %%time
    aksara=[26,27]
    aksara_num=[22,24]
    for i in range(0,2):
        gan = GAN()
        gan.train(aksara[i], aksara_num[i], epochs=30001, batch_size=60, sample_interval=1000)
        print("-----------------------------------------------------------------")
#     a = [0,2,4,5,6,7,12,13,18,19,20,25,26,27]
#     for i in a:
#         gan.train(aksara=i, epochs=30001, batch_size=32, sample_interval=2000)

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               401920    
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               131328    
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 256)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 257       
Total params: 533,505
Trainable params: 533,505
Non-trainable params: 

C:\Users\Sadrakh\AppData\Local\Continuum\anaconda3\lib\site-packages\keras\engine\training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


26-0 [D loss: 1.122073, acc.: 40.83%] [G loss: 0.868423]
26-1000 [D loss: 0.143286, acc.: 96.67%] [G loss: 3.289359]
26-2000 [D loss: 0.216810, acc.: 93.33%] [G loss: 2.531711]
26-3000 [D loss: 0.273464, acc.: 90.83%] [G loss: 2.181910]
26-4000 [D loss: 0.164020, acc.: 95.00%] [G loss: 2.515854]
26-5000 [D loss: 0.291590, acc.: 86.67%] [G loss: 2.534489]
26-6000 [D loss: 0.287588, acc.: 89.17%] [G loss: 2.754672]
26-7000 [D loss: 0.133445, acc.: 95.00%] [G loss: 2.677151]
26-8000 [D loss: 0.183777, acc.: 95.00%] [G loss: 3.026753]
26-9000 [D loss: 0.126500, acc.: 92.50%] [G loss: 4.036805]
26-10000 [D loss: 0.203842, acc.: 90.83%] [G loss: 4.285261]
26-11000 [D loss: 0.216017, acc.: 92.50%] [G loss: 3.144452]
26-12000 [D loss: 0.229879, acc.: 89.17%] [G loss: 3.129029]
26-13000 [D loss: 0.170899, acc.: 94.17%] [G loss: 4.159106]
26-14000 [D loss: 0.168630, acc.: 93.33%] [G loss: 4.212809]
26-15000 [D loss: 0.169313, acc.: 94.17%] [G loss: 4.976254]
26-16000 [D loss: 0.155719, acc.: 92.